In [2]:
import torch
import llm4ranking

from rich import print

In [3]:
# List supported reranking approaches

llm4ranking.list_available_reranking_approaches()

['rankgpt',
 'rel-gen',
 'query-gen',
 'prp-heap',
 'prp-allpair',
 'prp-bubble',
 'tourrank',
 'first',
 'fg-rel-gen']

In [4]:
# Load a reranker based on OpenAI API

from llm4ranking import Reranker

reranker = Reranker(
    reranking_approach="rankgpt",
    model_type="openai",
    model_name="gpt-4o",
    model_args={"api_key": "sk-xxxxx"}
)

print(reranker.LLM)
print(reranker.ranking_func)

<llm4ranking.lm.openai.OpenAIClient object at 0x7ff2b69a0ac0>

<llm4ranking.model.rankgpt.RankGPT object at 0x7ff2b69a30d0>

In [5]:
# Load a reranker based on HuggingFace model

reranker = Reranker(
    reranking_approach="rankgpt",
    model_type="hf",
    model_name="Qwen/Qwen2.5-7B-Instruct",
    model_args={"device_map": "cuda:0"}
)

print(reranker.LLM)
print(reranker.ranking_func)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

<llm4ranking.lm.huggingface.HFLM object at 0x7ff2b69c8970>

<llm4ranking.model.rankgpt.RankGPT object at 0x7ff2b69c87c0>

In [6]:
from datasets import load_dataset

data = load_dataset("json", data_files=f"../toy_data/dl19_bm25_top20.jsonl", split="train")
query = data[0]["query"]
candidates = [x["content"] for x in data[0]["hits"]]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
# A minimal example of reranking using the reranker

results, indices, records = reranker.rerank(
    query=query,
    candidates=candidates,
    return_indices=True,
    return_record=True,
)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [8]:
print(records)

RankingRecord(
    query='how long is life cycle of flea',
    candidates=None,
    prompt_template='I will provide you with {{ candidates|length }} passages, each indicated by a numerical 
identifier []. Rank the passages based on their relevance to the search query: {{ query }}.\n{% for content in 
candidates %}\n[{{ loop.index }}] {{content}}\n{% endfor %}\nSearch Query: {{ query }}.\nRank the {{ 
candidates|length }} passages above based on their relevance to the search query. All the passages should be 
included and listed using identifiers, in descending order of relevance. The output format should be [] > [] > ...,
e.g., [4] > [2] > ..., Only respond with the ranking results, do not say anything else or explain.\n',
    num_processed_docs=20,
    num_processed_tokens=2213,
    num_generated_tokens=86,
    latency=4.254306793212891,
    lm_outputs=[
        LMOuput(
            text='[2] > [1] > [3] > [10] > [18] > [7] > [11] > [13] > [4] > [19] > [20] > [5] > [6] > [12] > [14] >
[15] > [16] > [8] > [9]',
            loglikelihood=None,
            num_processed_tokens=2213,
            num_generated_tokens=86,
            logits=None
        )
    ],
    rank_indices=[1, 0, 2, 9, 17, 6, 10, 12, 3, 18, 19, 4, 5, 11, 13, 14, 15, 7, 8, 16]
)

In [9]:
del reranker
torch.cuda.empty_cache()

In [10]:
# Evaluation on TREC-DL19 benchmark

from llm4ranking.evaluation.evaluator import simple_evaluate

results = simple_evaluate(
    model_type="hf",
    model_args={"model": "Qwen/Qwen2.5-7B-Instruct", "device_map": "cuda:0"},
    datasets=["dl19"],
    reranking_approach="rankgpt",
    retriever="bm25",
    topk=20,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 43/43 [02:15<00:00,  3.16s/it]


MAP@1       	0.0261
MAP@5       	0.0865
MAP@10      	0.1305
MAP@20      	0.1798
MAP@100     	0.1798
NDCG@1      	0.7607
NDCG@5      	0.6641
NDCG@10     	0.6171
NDCG@20     	0.5458
NDCG@100    	0.3399
Recall@1    	0.0261
Recall@5    	0.0883
Recall@10   	0.1419
Recall@20   	0.2042
Recall@100  	0.2042


In [11]:
print(results)

{
    'dl19': {
        'pass0': {
            'metrics': {
                'MAP@1': 0.0261,
                'MAP@5': 0.0865,
                'MAP@10': 0.1305,
                'MAP@20': 0.1798,
                'MAP@100': 0.1798,
                'NDCG@1': 0.7607,
                'NDCG@5': 0.6641,
                'NDCG@10': 0.6171,
                'NDCG@20': 0.5458,
                'NDCG@100': 0.3399,
                'Recall@1': 0.0261,
                'Recall@5': 0.0883,
                'Recall@10': 0.1419,
                'Recall@20': 0.2042,
                'Recall@100': 0.2042
            },
            'records': [
                {
                    'query': 'how long is life cycle of flea',
                    'candidates': None,
                    'prompt_template': 'I will provide you with {{ candidates|length }} passages, each indicated by
a numerical identifier []. Rank the passages based on their relevance to the search query: {{ query }}.\n{% for 
content in candidates %}\n[{{ loop.index }}] {{content}}\n{% endfor %}\nSearch Query: {{ query }}.\nRank the {{ 
candidates|length }} passages above based on their relevance to the search query. All the passages should be 
included and listed using identifiers, in descending order of relevance. The output format should be [] > [] > ...,
e.g., [4] > [2] > ..., Only respond with the ranking results, do not say anything else or explain.\n',
                    'num_processed_docs': 20,
                    'num_processed_tokens': 2213,
                    'num_generated_tokens': 86,
                    'latency': 3.1397864818573,
                    'lm_outputs': [
                        {
                            'text': '[2] > [1] > [3] > [10] > [18] > [7] > [11] > [13] > [4] > [19] > [20] > [5] > 
[6] > [12] > [14] > [15] > [16] > [8] > [9]',
                            'loglikelihood': None,
                            'num_processed_tokens': 2213,
                            'num_generated_tokens': 86,
                            'logits': None
                        }
                    ],
                    'rank_indices': [1, 0, 2, 9, 17, 6, 10, 12, 3, 18, 19, 4, 5, 11, 13, 14, 15, 7, 8, 16]
                },
                {
                    'query': 'cost of interior concrete flooring',
                    'candidates': None,
                    'prompt_template': 'I will provide you with {{ candidates|length }} passages, each indicated by
a numerical identifier []. Rank the passages based on their relevance to the search query: {{ query }}.\n{% for 
content in candidates %}\n[{{ loop.index }}] {{content}}\n{% endfor %}\nSearch Query: {{ query }}.\nRank the {{ 
candidates|length }} passages above based on their relevance to the search query. All the passages should be 
included and listed using identifiers, in descending order of relevance. The output format should be [] > [] > ...,
e.g., [4] > [2] > ..., Only respond with the ranking results, do not say anything else or explain.\n',
                    'num_processed_docs': 20,
                    'num_processed_tokens': 2502,
                    'num_generated_tokens': 90,
                    'latency': 3.273911952972412,
                    'lm_outputs': [
                        {
                            'text': '[1] > [2] > [8] > [14] > [1] > [9] > [11] > [12] > [16] > [3] > [4] > [5] > 
[6] > [7] > [10] > [13] > [15] > [17] > [19] > [20]',
                            'loglikelihood': None,
                            'num_processed_tokens': 2502,
                            'num_generated_tokens': 90,
                            'logits': None
                        }
                    ],
                    'rank_indices': [0, 1, 7, 13, 8, 10, 11, 15, 2, 3, 4, 5, 6, 9, 12, 14, 16, 18, 19, 17]
                },
                {
                    'query': 'definition declaratory judgment',
                    'candidates': None,
                    'prompt_template': 'I will provide you